In [2]:
%matplotlib widget
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from pycpd import RigidRegistration, DeformableRegistration

In [4]:
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[1:3]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
graph_pos=[]
skels_aligned=[]
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    dilated = read_mat(path_snap+'/Analysis/dilated.mat')['dilated']
    skel = read_mat(path_snap+'/Analysis/skeleton.mat')['skeleton']
    dilateds.append(dilated)
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
#     nx_graph,pos = generate_nx_graph(from_sparse_to_graph(skel_doc))
#     graph_pos.append((nx_graph,pos))
#     print(len_connected)

In [14]:
def get_importance(g,node):
    score = np.inf
    for edge in g.edges(node):
        score =  min(score,g.get_edge_data(*edge)['weight'])
    return(score)
def realign_final(skeleton1,skeleton2):
    nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
    nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
    pruned1 = prune_graph(nx_graph1)
    pruned2 = prune_graph(nx_graph2)
    X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3 and get_importance(pruned1,node)>=10]))
    Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3 and get_importance(pruned2,node)>=10]))
    print(X.shape,Y.shape)
    reg = RigidRegistration(R=np.array([[1,0],[0,1]]),t=np.array([[0,0]]),**{'X': np.transpose(X), 'Y': np.transpose(Y)})
    out = reg.register()
    Rfound = reg.R
    tfound= reg.t
    skel_transformed = transform_skeleton(skeleton2,Rfound,tfound)
    skel_mat = np.zeros((26322, 49527),dtype=np.uint8)
    for pixel in skel_transformed.keys():
        skel_mat[pixel]=1
    return(skel_mat,Rfound,tfound)
def prune_graph(nx_graph,threshold=100):
    #should implement threshold!
    S = [nx_graph.subgraph(c).copy() for c in nx.connected_components(nx_graph)]
    selected = [g for g in S if g.size(weight="weight")*len(g.nodes)/10**6>=threshold]
    len_connected=[(nx_graph.size(weight="weight")*len(nx_graph.nodes)/10**6) for nx_graph in selected]
    print(len_connected)
    G = selected[0]
    for g in selected[1:]:
        G = nx.compose(G,g)
    return(G)
def transform_skeleton(skeleton_doc,Rot,trans):
    transformed_skeleton={}
    transformed_keys = (np.round(np.transpose(np.dot(np.linalg.inv(Rot),np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int))
    i=0
    for pixel in list(transformed_keys):
        i+=1
        transformed_skeleton[(pixel[0],pixel[1])]=1
    return(transformed_skeleton)

In [6]:
plt.close('all')

In [15]:
realigned = realign_final(skel_docs[0],skel_docs[1])

[700.88858]
[1012.7436]
(2, 1509) (2, 1786)


ValueError: operands could not be broadcast together with shapes (2,347476) (2,) 

In [ ]:
realigned

In [ ]:
plot_t_tp1([],[],None,None,realigned[0][19000:21000,44000:46000],skels[0][19000:21000,44000:46000])

In [56]:
plot_t_tp1([],[],None,None,skels[0][19000:21000,44000:46000],skels[1][19000:21000,44000:46000])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
pruned = [prune_graph(nx_graph_pos[0]) for nx_graph_pos in graph_pos]

[1581.43881]
[1993.99993]
[2555.631652]
[2893.02801]
[3334.03228]


In [ ]:
realigned=[skels[0]]
for i,date in enumerate(dates[:-1]):
    realigned,R,t = realign_final(realigned[-1],skel_docs[i+1])
    
    